# 基于增强超像素分割和注意力机制的海面溢油检测HMOSDN

- <font color="green">一.超像素:图像预处理技术,将具有相似特征的相邻像素聚集成更大的像素块,从而减少后续处理复杂度<br>
特点:<br>
1.局部同质性:每个超像素内部颜色,纹理等特征高度相似<br>
2.边界贴合性:较好地贴合图像真实边界<br>
3.降维作用:将数万像素简化为数百个超像素,降低计算量<br>   
</font><br>
- <font color="green">二.混合注意力机制<br>
1.光谱注意力模块:学习不同光谱波段的重要性,增强对目标敏感的光谱特征<br>
2.空间注意力模块:学习图像不同空间位置的重要性,突出目标区域<br>
</font>


# 第一遍
- 做了什么:提出融合改进超像素分割(ISLIC)和混合注意力机制(MAM)的海洋溢油检测网络HMOSDN
- 结果:在HOSD数据集上的实验表明:<br>
1.与传统机器学习方法相比,AUC值提升6.68%,平均召回率提升13.93%<br>


<img src="屏幕截图 2025-07-04 215219.png" width="300px"><br>
<font color="green">
1.AUC值:ROC曲线下面积,衡量分类模型区分正负样本的能力<br>
上图中,横轴为假阳性率FPR(实际为0,被识别为1),纵轴为真阳性率TPR(实际为1,被认为是1)<br>
2.召回率:衡量模型正确识别所有正样本的能力<br>
召回率=真正例(TP)/(真正例(TP)+假反例(FN))<br>
真正例:实际为1,识别为1<br>
假反例:实际为1,识别为0<br>
</font>

2.与深度学习方法相比,HMOSDN在多数图像上取得最佳检测结果<br>
3.消融实验证明ISLIC和MAM的有效性<br>
4.仅需1%的训练样本即可取得优异的检测成果

<font color="green"> 消融实验:系统性移除或修改模型的某些组件,评估这些组件对模型整体性能的贡献 </font>

## 关键图表
<img src="屏幕截图 2025-07-04 222140.png" width="800px"><br>
上图是HMOSDN的网络架构<br>
具体的实现步骤为:<br>
输入高光谱图像->主成分分析(PCA)降维->ISLIC(改进的简单线性迭代聚类)处理<br>
->特征堆叠->MAM-SSFEN模型处理


## 创新点与展望:
### 创新点
1.设计MAM-SSFEN网络,融合光谱-空间注意力,强化关键特征聚焦能力<br>
2.ISLIC算法的提出:通过GF和PIST优化超像素分割,提升空间特征提取鲁棒性
### 展望:
1.引入光谱信息时序特征,提升动态检测能力<br>
2.基于新型网络的检测框架,平衡精度与效率<br>
3.结合无人机,卫星等多元数据,构建多尺度溢油监测体系(这个就算了)<br>

# 第二遍

<img src="屏幕截图 2025-07-04 222140.png" width="400px"><br>
## 还是从HMOSDN的网络架构说起


### 1.主成分分析降维:通过线性变换将高光谱数据投影到低维空间.步骤:<br>
(1).数据中心化:对原始数据矩阵X,计算各个波段的均值并减去,得到中心化矩阵<br>
(2).协方差矩阵计算:<br>
$$ C=\frac{1}{n-1}X^TX $$
其中,c为协方差矩阵,x为中心化后的数据矩阵分母为n-1为了无偏估计<br>
(3).特征值与特征向量求解:关键公式:<br>
$$C=VEV^T$$
其中 V是特征向量矩阵(列向量为特征向量)，E是对角矩阵(对角线为特征值)<br>
4.低维特征投影:按特征值从大到小排序,选取前k个主成分对应的特征向量组成矩阵<br>
关键公式:
$$F=V_kX$$
其中,Vk矩阵是前k个特征向量组成的矩阵,F是降维后的特征图像

### ISLIC(improved simple linear iterative clustering)改进的简单线性迭代聚类
#### 什么是聚类
- 聚类是一种无监督学习方法，其核心目标是将一组数据对象按照某种相似性度量标准划分为若干个簇（Cluster），<br>使得同一簇内的数据对象之间的相似性较高，而不同簇中的数据对象之间的相似性较低。
#### 什么是简单线性迭代聚类
1.初始化种子点:按照设定的超像素个数，在图像内均匀分配种子点。假设图像总共有N个像素点，预分割为K个相同尺寸的超像素，那么每个超像素的大小为N/K，则相邻种子点的距离（步长）近似为$\sqrt{N/k}$<br>
2.重新选择种子点:在种子点的3x3邻域内重新选择种子点，计算该邻域内所有像素点的梯度值，将种子点移到梯度最小的地方，以避免种子点落在梯度较大的轮廓边界上<br>
3.在每个种子点周围的2Sx2S邻域内为每个像素点分配类标签，计算每个像素点与种子点的颜色距离和空间距离，取最小值对应的种子点作为该像素点的聚类中心<br>
4.上述步骤不断迭代，直到误差收敛，即每个像素点的聚类中心不再发生变化。<br>
#### 文中对ISLIC的改进
##### 1.高斯滤波平滑:
利用基于高斯分布的线性滤波器对图像进行了加权平均<br>
关键公式为:

$$G(X,Y)=\frac{1}{2\pi\sigma ^2 }e^{-\frac{x^2+y^2}{2\sigma^2}}$$
其中$\sigma$为标准差,用于决定平滑程度

对于像素点$(x_p,y_p)$,高斯平滑公式为:
$$\hat{f_p}=\sum_{(x_q,y_q)\in U_p} G(x_p-x_q,y_p-y_q)f_q$$
其中,$U_p$指的是p点的邻域

##### 2.像素强度平滑技术
计算每个超像素内所有像素的平均强度用以替代该超像素内所有像素的强度值消除局部噪声

### 特征堆叠
- 光谱特征:通过pca主成分分析从原始高光谱图像中提取关键光谱信息,生成降维后的图像,获取光谱特征
- 空间特征:通过ISLIC算法对降维后的图像进行超像素分割,获取空间特征
- 堆叠方式:在空间维度上拼接(长宽均一致,高相加)

### 3d卷积
$$Y_{i,j,l,f}=\sum_{m=0}^{k-1}\sum_{n=0}^{k-1}\sum_{p=0}^{k-1}\sum_{c=1}^{C}W_{m,n,p,c,f} \cdot  X_{i+m,j+n,l+p,c}+b_f$$
其中,i,j,l是输出位置的坐标,$b_f$是偏置项

### MAM
<img src="屏幕截图 2025-07-08 144907.png" width="300px"><br>
- MAM由两个串联的子模块组成,顺序为光谱注意力->空间注意力(实验表明该顺序更优)
#### 1.输入特征图$F \in \mathbb{R}^{H*W*D*C} $ <br>


<font color="green">
特征图的四个维度:
batch样本个数*空间高度*空间宽度*通道维数
</font>

#### 2.光谱注意力模块
1.对输入的特征图F的每个光谱带分别执行全局最大池化和平均池化,得到两种池化特征<br>
2.将两种池化特征输入共享mlp(多层感知机),学习各个光谱带来的注意力权重<br>
3.对mlp输出的两个特征图进行元素相加,通过sigmoid函数将权重限制在0-1之间,生成光谱注意力矩阵<br>
4.通过元素-wise乘法,将光谱注意力权重与原始特征图融合得到增强光谱特征图$\hat{F}$


<font color="green">
元素-wise乘法:同型矩阵点积
</font>

#### 3.空间注意力模块
1.将光谱增强后的特征图$/hat{F}$在光谱维度进行全局最大化和平均池化,得到两个空间特征图<br>
2.将两个空间特征图沿光谱维度拼接,通过卷积操作生成空间注意力权重<br>
3.经过sigmoid激活函数后得到空间注意力矩阵$M_{spa}(\hat{F})$<br>
4.通过元素-wise乘法,将空间注意力权重与$/hat{F}$融合,得到最终的光谱-空间增强特征图$\hat{\hat{F}}$

#### 输出
增强特征图传入后续网络在进行进一步特征提取